In [1]:
# -*- coding: utf-8 -*-
import xlrd    #xlrd是读excel
import numpy
import time
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.model_selection import LeaveOneOut
import math
import numpy.linalg as LA
from sklearn.cluster import KMeans
import random
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import model_selection,metrics
from sklearn.model_selection import GridSearchCV
import matplotlib.pylab as plt
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from scipy import interp
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
random.sample
import pandas as pd
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from collections import Counter
startTime = time.time()
a=open('./data/Known disease-miRNA association number.xlsx')
b=open('./data/Disease semantic similarity matrix-model 1.xlsx')
c=open('./data/Disease semantic similarity matrix-model 2.xlsx')
d=open('./data/Disease semantic similarity weighting matrix.xlsx')
e=open('./data/miRNA functional similarity matrix.xlsx')
f=open('./data/miRNA functional similarity weighting matrix.xlsx')
g=open('./data/miRNA number.xlsx')
h=open('./data/disease number.xlsx')

In [2]:
SS=numpy.zeros((383,383))  #383*383阶的零矩阵
A=numpy.zeros((383,495))
KD=numpy.zeros((383,383))
SD=numpy.zeros((383,383))
DWM=numpy.zeros((383,383))
MWM=numpy.zeros((495,495))
FS=numpy.zeros((495,495))
SM=numpy.zeros((495,495))
KM=numpy.zeros((495,495))

In [3]:
xlsx1=xlrd.open_workbook('./data/Disease semantic similarity matrix-model 1.xlsx')
xlsx2=xlrd.open_workbook('./data/Disease semantic similarity matrix-model 2.xlsx')  #打开Excel文件读取数据
sheet1=xlsx1.sheets()[0]        ##通过索引顺序获取
sheet2=xlsx2.sheets()[0]
for i in range(383):
    for j in range(383):
        s1=sheet1.row_values(i)  #获取整行和整列的值（数组）
        s2=sheet2.row_values(i)
        m=s1[j]
        n=s2[j]
        SS[i,j]=float(m+n)/2          

In [4]:
xlsx3=xlrd.open_workbook('./data/Known disease-miRNA association number.xlsx')
sheet3=xlsx3.sheets()[0]
for i in range(5430):
    s3=sheet3.row_values(i)
    m=int(s3[0])
    n=int(s3[1])
    A[n-1,m-1]=1                    #Obtain adjacency matrix A   邻接矩阵  行：疾病 列：miRNA
xlsx4=xlrd.open_workbook('./data/Disease semantic similarity weighting matrix.xlsx')
sheet4=xlsx4.sheets()[0]
for i in range(383):
    for j in range(383):
        s4=sheet4.row_values(i)
        DWM[i,j]=s4[j]               #Get disease semantic weighting matrix DJQ
xlsx5=xlrd.open_workbook('./data/miRNA functional similarity weighting matrix.xlsx')
sheet5=xlsx5.sheets()[0]
for i in range(495):
    for j in range(495):
        s5=sheet5.row_values(i)
        MWM[i,j]=s5[j]               #Get miRNA functional similarity weighting matrix MJQ
xlsx6=xlrd.open_workbook('./data/miRNA functional similarity matrix.xlsx')
sheet6=xlsx6.sheets()[0]
for i in range(495):
    for j in range(495):
        s6=sheet6.row_values(i)
        FS[i,j]=s6[j]                  #Get miRNA functional similarity matrix FS

In [5]:
C=np.asmatrix(A)           #矩阵C   asmatrix(data[, dtype])：将输入解释为矩阵.
gamd=383/(LA.norm(C,'fro')**2);   #norm(X,'fro') 计算矩阵 (向量) C的F范数;F范数是把一个矩阵中每个元素的平方求和后开根号；**两个乘号就是乘方，比如2**4,结果就是2的4次方，结果是16
kd=np.mat(np.zeros((383,383)))     #创建一个零矩阵
km=np.mat(np.zeros((495,495)))
D=C*C.T;          #C*C的转置矩阵
for i in range(383):
        for j in range(i,383):
            kd[j,i]=np.exp(-gamd*(D[i,i]+D[j,j]-2*D[i,j]))   # 高斯核相似计算
kd=kd+kd.T-np.diag(np.diag(kd))   #两次使用diag() 获得某二维矩阵的对角矩阵：
KD=np.asarray(kd)                  #Obtain Gaussian interaction profile kernel similarity for disease SD
kd=[]
SD = np.multiply(SS,DWM)+np.multiply(KD,(1-DWM))     #np.multiply(A,B)矩阵对应元素位置相乘   疾病综合相似性
SD=np.asarray(SD)                                   #DWM  is disease semantic weighting matrix
gamam = 495/(LA.norm(C,'fro')**2);
E=C.T*C;
for i in range(495):
    for j in range(i,495):
        km[i,j]=np.exp(-gamam*(E[i,i]+E[j,j]-2*E[i,j]))
km=km+km.T-np.diag(np.diag(km))
KM=np.asarray(km)
km=[]
SM = np.multiply(FS,MWM)+np.multiply(KM,(1-MWM))
SM=np.asarray(SM)                                         #Obtain Gaussian interaction profile kernel similarity for miRNA SM

In [6]:
#K_mean clustering
unknown=[]
known=[]
for x in range(383):
    for y in range(495):
        if A[x,y]==0:
            unknown.append((x,y))    #append() 方法向列表的尾部添加一个新的元素
        else:
            known.append((x,y))               #Divide the samples into Major and Minor
            
# if balanced:
#     unknown = random.sample(unknown, len(known))

# negative_x = [] # 负样本
# for z in range(len(unknown)):
#     q=SD[unknown[z][0],:].tolist()+SM[unknown[z][1],:].tolist()#tolist()将数组或者矩阵转换成列表    X[1,:] 取第一行的所有列数据
#     negative_x.append(q)

# positive_x = [] # 正样本
# for z in range(len(known)):
#     q=SD[known[z][0],:].tolist()+SM[known[z][1],:].tolist()#tolist()将数组或者矩阵转换成列表    X[1,:] 取第一行的所有列数据
#     positive_x.append(q)

In [7]:
positive = pd.DataFrame(known, columns = ['disease', 'miRNA'])
negative = pd.DataFrame(unknown, columns = ['disease', 'miRNA'])
positive['label'] = 1
negative['label'] = 0
dtp = pd.concat([positive, negative], axis = 0)
dtp

,disease,miRNA,label
0,0,0,1
1,0,1,1
2,0,2,1
3,1,3,1
4,1,4,1
...,...,...,...
184150,382,490,0
184151,382,491,0
184152,382,492,0
184153,382,493,0


In [8]:
SD = pd.DataFrame(SD)
SM = pd.DataFrame(SM)

# run

In [9]:
def sample(all_associations, random_seed):
    
    known_associations = all_associations.loc[all_associations['label'] == 1]
    unknown_associations = all_associations.loc[all_associations['label'] == 0]
    random_negative = unknown_associations.sample(n = known_associations.shape[0], random_state = random_seed, axis=0)

    sample_df = known_associations.append(random_negative)
    sample_df.reset_index(drop=True, inplace=True)

    return sample_df

In [10]:
def obtain_data(dtp, ID, IM, isbalance):

    if isbalance:
        dtp = sample(dtp, random_seed = 1234)

    mirna_ids = list(set(dtp['miRNA']))
    disease_ids = list(set(dtp['disease']))
    random.shuffle(mirna_ids)
    random.shuffle(disease_ids)
    print('# miRNA = {} | Disease = {}'.format(len(mirna_ids), len(disease_ids)))

    mirna_test_num = int(len(mirna_ids) / 5)
    disease_test_num = int(len(disease_ids) / 5)
    print('# Test: miRNA = {} | Disease = {}'.format(mirna_test_num, disease_test_num))    
    
    samples = pd.merge(pd.merge(dtp, ID, left_on = 'disease', right_index = True), IM, left_on = 'miRNA', right_index = True).reset_index(drop = True)
#     samples.drop(labels = ['id_x', 'id_y'], axis = 1, inplace = True)
    
    return mirna_ids, disease_ids, mirna_test_num, disease_test_num, samples


In [11]:
def performances(y_true, y_pred, y_prob):

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()

    accuracy = (tp+tn)/(tn+fp+fn+tp)
    
    if tp+fn == 0:
        recall = 0
    else:
        recall = tp / (tp+fn)
    
    if tp+fp == 0:
        precision = 0
    else:
        precision = tp / (tp+fp)
    
    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2*precision*recall / (precision+recall)
    
    roc_auc = roc_auc_score(y_true, y_prob)
    prec, reca, _ = precision_recall_curve(y_true, y_prob)
    aupr = auc(reca, prec)
    
    print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
    print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
    print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
    print('acc={:.4f}|precision={:.4f}|recall={:.4f}|f1={:.4f}|auc={:.4f}|aupr={:.4f}'.format(accuracy, precision, recall, f1, roc_auc, aupr))
    return (y_true, y_pred, y_prob), (accuracy, precision, recall, f1, roc_auc, aupr)


In [12]:
def generate_task_Tp_train_test_idx(samples):
    kf = KFold(n_splits = 5, shuffle = True, random_state = 1234)

    train_index_all, test_index_all, n = [], [], 0
    train_id_all, test_id_all = [], []
    fold = 0
    for train_idx, test_idx in tqdm(kf.split(samples.iloc[:, 3:])): #train_index与test_index为下标
        print('-------Fold ', fold)
        train_index_all.append(train_idx) 
        test_index_all.append(test_idx)

        train_id_all.append(np.array(dtp.iloc[train_idx][['miRNA', 'disease']]))
        test_id_all.append(np.array(dtp.iloc[test_idx][['miRNA', 'disease']]))

        print('# Pairs: Train = {} | Test = {}'.format(len(train_idx), len(test_idx)))
        fold += 1
    return train_index_all, test_index_all, train_id_all, test_id_all


# In[7]:

def generate_task_Tm_Td_train_test_idx(item, ids, dtp):
    
    test_num = int(len(ids) / 5)
    
    train_index_all, test_index_all = [], []
    train_id_all, test_id_all = [], []
    
    for fold in range(5):
        print('-------Fold ', fold)
        if fold != 4:
            test_ids = ids[fold * test_num : (fold + 1) * test_num]
        else:
            test_ids = ids[fold * test_num :]

        train_ids = list(set(ids) ^ set(test_ids))
        print('# {}: Train = {} | Test = {}'.format(item, len(train_ids), len(test_ids)))

        test_idx = dtp[dtp[item].isin(test_ids)].index.tolist()
        train_idx = dtp[dtp[item].isin(train_ids)].index.tolist()
        random.shuffle(test_idx)
        random.shuffle(train_idx)
        print('# Pairs: Train = {} | Test = {}'.format(len(train_idx), len(test_idx)))
        assert len(train_idx) + len(test_idx) == len(dtp)

        train_index_all.append(train_idx) 
        test_index_all.append(test_idx)
        
        train_id_all.append(train_ids)
        test_id_all.append(test_ids)
        
    return train_index_all, test_index_all, train_id_all, test_id_all


In [13]:
def transfer(y_pred):
    return [[0,1][x>0.5] for x in y_pred.reshape(-1)]

In [14]:
def run(train_index_all, test_index_all, samples):
    
    fold = 0
    for train_idx, test_idx in zip(train_index_all, test_index_all):
        print('----------------------- Fold = ', str(fold))

        X = samples.iloc[:, 3:]
        y = samples['label']
        x_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        x_test, y_test = X.iloc[test_idx], y.iloc[test_idx]
    
        GBDT=GradientBoostingClassifier(n_estimators = 12, max_depth = 5, min_samples_leaf = 13)
        # 训练模型
        GBDT.fit(x_train, y_train)
        OHE = OneHotEncoder()
        OHE.fit(GBDT.apply(x_train)[:, :, 0]) # model.apply(X_train)返回训练数据X_train在训练好的模型里每棵树中所处的叶子节点的位置（索引）
        LR = LogisticRegression(n_jobs = -1)
        LR.fit(OHE.transform(GBDT.apply(x_train)[:, :, 0]),y_train)
        
        y_train_prob = LR.predict_proba(OHE.transform(GBDT.apply(x_train)[:, :, 0]))[:,1]
        y_test_prob = LR.predict_proba(OHE.transform(GBDT.apply(x_test)[:, :, 0]))[:,1]

        y_train_pred, y_test_pred = transfer(y_train_prob), transfer(y_test_prob)

        print(len(y_train_pred), len(y_train))
        performances_train = performances(y_train, y_train_pred, y_train_prob)
        performances_test = performances(y_test, y_test_pred, y_test_prob)

        fold += 1
    
    return y_train_pred, y_test_pred, y_train_prob, y_test_prob, performances_train, performances_test


In [15]:
# mirna_ids, disease_ids, mirna_test_num, disease_test_num, samples = obtain_data(dtp, SD, SM, isbalance = True)
# item = 'miRNA'
# ids = mirna_ids
# train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, samples)
# fold = 0
# for train_idx, test_idx in zip(train_index_all, test_index_all):
#     print('----------------------- Fold = ', str(fold))

#     X = samples.iloc[:, 3:]
#     y = samples['label']
#     x_train, y_train = X.iloc[train_idx], y[train_idx]
#     x_test, y_test = X.iloc[test_idx], y[test_idx]

#     GBDT=GradientBoostingClassifier(n_estimators = 12, max_depth = 5, min_samples_leaf = 13)
#     # 训练模型
#     GBDT.fit(x_train, y_train)
#     OHE = OneHotEncoder()
#     OHE.fit(GBDT.apply(x_train)[:, :, 0]) # model.apply(X_train)返回训练数据X_train在训练好的模型里每棵树中所处的叶子节点的位置（索引）
#     LR = LogisticRegression(n_jobs = -1)
#     LR.fit(OHE.transform(GBDT.apply(x_train)[:, :, 0]),y_train)

#     y_train_prob = LR.predict_proba(OHE.transform(GBDT.apply(x_train)[:, :, 0]))[:,1]
#     y_test_prob = LR.predict_proba(OHE.transform(GBDT.apply(x_test)[:, :, 0]))[:,1]

#     y_train_pred, y_test_pred = transfer(y_train_prob), transfer(y_test_prob)

#     print(len(y_train_pred), len(y_train))
#     performances_train = performances(y_train, y_train_pred, y_train_prob)
#     performances_test = performances(y_test, y_test_pred, y_test_prob)
#     break

# Tp

In [16]:
for isbalance in [False, True]:
    
    mirna_ids, disease_ids, mirna_test_num, disease_test_num, samples = obtain_data(dtp, SD, SM, isbalance)
    for task in ['Tp', 'Tm', 'Td']:
        
        print('========== isbalance = {} | task = {}'.format(isbalance, task))
        
        if task == 'Tp':
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tp_train_test_idx(samples)
            
        elif task == 'Tm':
            item = 'miRNA'
            ids = mirna_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, samples)

        elif task == 'Td':
            item = 'disease'
            ids = disease_ids
            train_index_all, test_index_all, train_id_all, test_id_all = generate_task_Tm_Td_train_test_idx(item, ids, samples)

        y_train_pred, y_test_pred, y_train_prob, y_test_prob, performances_train, performances_test = run(train_index_all, test_index_all, samples)



# miRNA = 495 | Disease = 383
# Test: miRNA = 99 | Disease = 76


5it [00:00, 81.55it/s]

========== isbalance = False | task = Tp
-------Fold  0
# Pairs: Train = 151668 | Test = 37917
-------Fold  1
# Pairs: Train = 151668 | Test = 37917
-------Fold  2
# Pairs: Train = 151668 | Test = 37917
-------Fold  3
# Pairs: Train = 151668 | Test = 37917
-------Fold  4
# Pairs: Train = 151668 | Test = 37917
----------------------- Fold =  0



/home/chujunyi/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  


151668 151668
tn = 146766, fp = 527, fn = 2855, tp = 1520
y_pred: 0 = 149621 | 1 = 2047
y_true: 0 = 147293 | 1 = 4375
acc=0.9777|precision=0.7426|recall=0.3474|f1=0.4734|auc=0.9492|aupr=0.5614
tn = 36706, fp = 156, fn = 736, tp = 319
y_pred: 0 = 37442 | 1 = 475
y_true: 0 = 36862 | 1 = 1055
acc=0.9765|precision=0.6716|recall=0.3024|f1=0.4170|auc=0.9425|aupr=0.4851
----------------------- Fold =  1


...

/home/chujunyi/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  


8396 8396
tn = 3877, fp = 480, fn = 396, tp = 3643
y_pred: 0 = 4273 | 1 = 4123
y_true: 0 = 4357 | 1 = 4039
acc=0.8957|precision=0.8836|recall=0.9020|f1=0.8927|auc=0.9629|aupr=0.9577
tn = 890, fp = 183, fn = 219, tp = 1172
y_pred: 0 = 1109 | 1 = 1355
y_true: 0 = 1073 | 1 = 1391
acc=0.8369|precision=0.8649|recall=0.8426|f1=0.8536|auc=0.9088|aupr=0.9261
----------------------- Fold =  3


/home/chujunyi/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  


9228 9228
tn = 3819, fp = 544, fn = 371, tp = 4494
y_pred: 0 = 4190 | 1 = 5038
y_true: 0 = 4363 | 1 = 4865
acc=0.9008|precision=0.8920|recall=0.9237|f1=0.9076|auc=0.9625|aupr=0.9640
tn = 957, fp = 110, fn = 168, tp = 397
y_pred: 0 = 1125 | 1 = 507
y_true: 0 = 1067 | 1 = 565
acc=0.8297|precision=0.7830|recall=0.7027|f1=0.7407|auc=0.8999|aupr=0.8245
----------------------- Fold =  4
8929 8929
tn = 3768, fp = 532, fn = 363, tp = 4266
y_pred: 0 = 4131 | 1 = 4798
y_true: 0 = 4300 | 1 = 4629
acc=0.8998|precision=0.8891|recall=0.9216|f1=0.9051|auc=0.9645|aupr=0.9652
tn = 971, fp = 159, fn = 203, tp = 598
y_pred: 0 = 1174 | 1 = 757
y_true: 0 = 1130 | 1 = 801
acc=0.8125|precision=0.7900|recall=0.7466|f1=0.7677|auc=0.8917|aupr=0.8497


/home/chujunyi/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  
